# WIP: Isolation Forest

Isolation forest is an unsupervised training algorithm to identify outliers / anomalies in data. The algorithm works in the following way:

1. The data is randomly and recuresively partitioned and represented as a trees (forest).